# Cap 1: Wikipedia

## 1.6 Uma rede complexa real: Wikipédia

In [1]:
import networkx as nx
import wikipedia # a instalar
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# A linguagem de Wikipedia : português
wikipedia.set_lang("pt")

In [3]:
# no raiz
RAIZ = "Fortaleza".title()

In [4]:
# páginas excluidas
EXCL = ("International Standard Serial Number",
"International Standard Book Number",
"National Diet Library",
"International Standard Name Identifier",
"International Standard Book Number (Identifier)",
"Pubmed Identifier", 
"Pubmed Central",
"Digital Object Identifier", 
"Arxiv",
"Proc Natl Acad Sci Usa", 
"Bibcode",
"Library Of Congress Control Number", 
"Jstor",
"OpenStreetMap",
"Wikimapia",
"Instituto Nacional de Meteorologia",
"Instituto Brasileiro de Geografia e Estatística",
"Coeficiente de Gini"
)

In [5]:
# lista de nos por explorar
pend_lst = [(0, RAIZ)] # O vértice RAIZ está na camada 0

In [6]:
# conjunto de nos que ainda devem ser explorados
pend_cnj = set(RAIZ) # Só RAIZ

In [7]:
# conjunto de nos que ja foram explorados
feito_cnj = set() # Nehum nó explorado

In [8]:
F = nx.DiGraph()

In [9]:
camada, pag = pend_lst[0]

In [10]:
while camada < 2:
    del pend_lst[0]
    feito_cnj.add(pag)
    print(camada, pag)
    try:
        wiki = wikipedia.page(pag) 
        # pode ser carregada a pagina?
        wiki.links 
        # bug não dá certo se a página não tem links
    except:
        camada, pag = pend_lst[0]
        print('Nao pude carregar', pag)
        continue
    for link in wiki.links:
        link = link.title()
        if link not in EXCL and not link.startswith("Lista de"):
            if link not in pend_cnj and link not in feito_cnj:
                pend_lst.append((camada + 1, link))
                pend_cnj.add(link)
            F.add_edge(pag, link)
    camada, pag = pend_lst[0]

0 Fortaleza
1 13 De Abril
1 15 De Agosto
1 1726
1 1884
1 19 De Março
1 1 De Abril
1 2019 No Brasil
1 25 De Março
1 89 Fm
1 8 De Dezembro
1 Am Cidade
1 A Igreja De Jesus Cristo Dos Santos Dos Últimos Dias
1 Abolicionismo No Brasil
1 Abril Despedaçado (Filme)
1 Academia Brasileira De Letras
1 Academia Cearense De Letras
1 Academia De Letras
1 Acaraú
1 Acidente Rodoviário
1 Acopiara
1 Acorizal
1 Acre
1 Adolfo Caminha
1 Adolfo Herbster
1 Aeroporto Internacional Pinto Martins
1 Aeroporto Internacional De Fortaleza
1 Agricultura
1 Agropecuária
1 Alagoas
1 Alberto Nepomuceno
1 Alcântara (Maranhão)
1 Aldeamento
1 Aldeota
1 Alfabetização
1 Algodão
1 Alimento
1 Almas (Tocantins)
1 Aloísio Lorscheider
1 Alta Cultura
1 Altitude
1 Alvorada (Rio Grande Do Sul)
1 Amapá
1 Amarelos
1 Amazonas
1 Amazônia
1 Amelinha
1 América Latina
1 Ananindeua
1 Andaraí
1 Angra Dos Reis
1 Antonina
1 António José Da Silva Paulet
1 Antônio Bezerra (Fortaleza)
1 Antônio Cambraia
1 Antônio Pinto Nogueira Acioly
1 Anápolis


In [ ]:
print("{} nos, {} arestas".format(F.order(), nx.number_of_edges(F)))

In [ ]:
# Salva a rede
with open('wikifortaleza.p', 'wb') as f:
    pickle.dump(F, f)

In [ ]:
# Apagamos laços
F.remove_edges_from(nx.selfloop_edges(F))
print("{} nós , {} arestas".format(F.order(), nx.number_of_edges(F)))

In [ ]:
# Eliminamos duplicados - Plurais
duplicados = [(vert, vert + "s") for vert in F if vert + "s" in F]
for dup in duplicados:
    F = nx.contracted_nodes(F, *dup, self_loops = False)
    
print("{} nós, {} arestas".format(F.order(), nx.number_of_edges(F)))

In [ ]:
# Eliminamos duplicados - " - "
duplicados = [(x,y) for x , y in [(vert, vert.replace("-", " ")) for vert in F] if x!=y and y in F]
for dup in duplicados:
    F = nx.contracted_nodes(F, *dup, self_loops=False)
    
print("{} nós, {} arestas".format(F.order(), nx.number_of_edges(F)))

In [ ]:
# Salva a rede
with open("wikifortaleza - semdup.p", 'wb') as f:
    pickle.dump (F , f )

In [ ]:
nx.draw(F, with_labels = "True")

In [ ]:
print("Densidade : ", nx.density(F))
print("Coeficiente de agrupamento : ", nx.transitivity(F))
print("Reciprocidade :" , nx.reciprocity(F))
hgrauF = nx.degree_histogram(F)
grausF = np.linspace(0 , len(hgrauF), len(hgrauF))

In [ ]:
# Networkx ainda não tem uma função como degree_histogram () para arestas
# saintes ou entrantes .
hgrauinF = [0]*len(hgrauF)
for ver, grau in F.in_degree():
    hgrauinF[grau] = hgrauinF[grau] + 1
    hgrauouF = [0]*len(hgrauF)
for ver, grau in F.out_degree():
    hgrauouF[grau] = hgrauouF[grau] + 1

In [ ]:
plt.figure()
plt.loglog(grausF,hgrauF,'b*',label='Grau')
plt.loglog(grausF,hgrauinF,'r^',label='Grau entrante')
plt.loglog(grausF,hgrauouF,'k+',label='Grau sainte')
plt.xlabel('grau')
plt.ylabel('frequência')
plt.legend()
plt.grid()

In [ ]:
nucleo = [vert for vert, grau in F.degree() if grau >= 10]
N = nx.subgraph(F, nucleo)

In [ ]:
# Elimina o atributo "contraction" dos nos
for (n,d) in N.nodes(data=True):
    try:
        del d["contraction"]
    except:
        continue

In [ ]:
# Elimina todos os atributos das arestas
for n1, n2, d in N.edges(data=True):
    d.clear()

In [ ]:
print("{} nos, {} arestas".format(len(N), nx.number_of_edges(N)))
nx.write_graphml(N, "wiki-fortaleza.graphml")